In [1]:
# Rasdaman Data Access - First Tries

In [5]:
import xmltodict as xmltodict
# get it with: pip install python-dotenv
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [20]:
import os
# read the secret information
rasdaman_username = os.getenv("RASDAMAN_CRED_USERNAME")
rasdaman_password = os.getenv("RASDAMAN_CRED_PASSWORD")
rasdaman_endpoint = os.getenv("RASDAMAN_SERVICE_ENDPOINT")

base_wcs_url = rasdaman_endpoint + "?service=WCS&version=2.1.0"

## WCS

In [30]:
import requests
import xmltodict

# describe the WCS
response = requests.get(base_wcs_url + "&request=GetCapabilities", auth=(rasdaman_username, rasdaman_password))
wcs_data = xmltodict.parse(response.content)
wcs_data

{'wcs:Capabilities': {'@version': '2.1.0',
  '@xsi:schemaLocation': 'http://www.opengis.net/wcs/2.1 http://schemas.opengis.net/wcs/2.1/wcsGetCapabilities.xsd',
  '@xmlns:wcs': 'http://www.opengis.net/wcs/2.1',
  '@xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance',
  '@xmlns:gml': 'http://www.opengis.net/gml/3.2',
  '@xmlns:swe': 'http://www.opengis.net/swe/2.0',
  '@xmlns:ows': 'http://www.opengis.net/ows/2.0',
  '@xmlns:xlink': 'http://www.w3.org/1999/xlink',
  '@xmlns:rasdaman': 'http://www.rasdaman.org',
  '@xmlns:int': 'http://www.opengis.net/wcs/interpolation/1.0',
  '@xmlns:crs': 'http://www.opengis.net/wcs/service-extension/crs/1.0',
  '@xmlns:inspire_dls': 'http://inspire.ec.europa.eu/schemas/inspire_dls/1.0',
  '@xmlns:inspire_common': 'http://inspire.ec.europa.eu/schemas/common/1.0',
  'ows:ServiceIdentification': {'ows:Title': 'rasdaman',
   'ows:Abstract': 'rasdaman server - free download from www.rasdaman.org',
   'ows:ServiceType': 'OGC WCS',
   'ows:ServiceTypeVers

In [31]:
# list the WCS contents
wcs_contents = wcs_data['wcs:Capabilities']['wcs:Contents']['wcs:CoverageSummary']
wcs_contents

{'wcs:CoverageId': 'imperviousness',
 'wcs:CoverageSubtype': 'RectifiedGridCoverage',
 'ows:WGS84BoundingBox': {'ows:LowerCorner': '-27.3444 27.0428',
  'ows:UpperCorner': '35.4948 72.2158'},
 'ows:BoundingBox': {'@crs': 'http://localhost:8080/rasdaman/def/crs/EPSG/0/4326',
  '@dimensions': '2',
  'ows:LowerCorner': '27.04280000000000 -27.3444',
  'ows:UpperCorner': '72.2158 35.4948'},
 'ows:AdditionalParameters': {'ows:AdditionalParameter': [{'ows:Name': 'sizeInBytes',
    'ows:Value': '283863518160'},
   {'ows:Name': 'sizeInBytesWithPyramidLevels', 'ows:Value': '307495836610'},
   {'ows:Name': 'axisList', 'ows:Value': 'Lat,Lon'}]}}

In [121]:
cov_id = "&COVERAGEID=" + wcs_contents['wcs:CoverageId']
encode_format = "&FORMAT=GEOTIFF_16"

subset_long = "&subset=Long(35.3, 35.4)"
subset_lat = "&subset=Lat(36.0, 37.0)"

response = requests.get(
    base_wcs_url + "&request=GetCoverage" + cov_id + subset_long + subset_lat + encode_format,
    auth=(rasdaman_username, rasdaman_password),
    verify=False)

with open('norway_imperviousness.tif', 'wb') as file:
    file.write(response.content)

import rasterio
img = rasterio.open('norway_imperviousness.tif', driver="GTiff")

from rasterio import plot
plot.show(img, cmap='gist_ncar')

RasterioIOError: 'norway_imperviousness.tif' not recognized as a supported file format.

In [118]:
from owslib.util import Authentication
from owslib.wcs import WebCoverageService

print(base_wcs_url)

wcs = WebCoverageService(
    'http://167.235.152.255:8080/rasdaman/ows?service=WCS&version=2.1.0',
    auth=Authentication(username=rasdaman_username, password=rasdaman_password),
    version='1.0.0'
)

print(list(wcs.contents))

http://167.235.152.255:8080/rasdaman/ows?service=WCS&version=2.1.0


ServiceException: <?xml version="1.0" encoding="UTF-8"?>
<ExceptionReport version="1.1.0" xmlns="http://www.opengis.net/ows/1.1"
  xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
  xsi:schemaLocation="http://www.opengis.net/ows/1.1 http://geowebcache.org/schema/ows/1.1.0/owsExceptionReport.xsd">
  <Exception exceptionCode="InvalidParameterValue" locator="request">
    <ExceptionText>Operation request contains an invalid parameter value</ExceptionText>    
  </Exception>
</ExceptionReport>

